<a href="https://colab.research.google.com/github/Zahan15/483_final_project/blob/master/483_Final_Proj_Test_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GNN for link prediction and graph classification task

Last time we explore a standard benchmark datase Cora, and implement a classic graph neural network GCN(Kipf et al. (2017)) for node classification task. In this Colab, we are going to explore two kinds of graph learning task: **link prediction** and **graph classification**. We will apply GCN to both of these two tasks. All of these implementations are still based on the [PyG](https://pytorch-geometric.readthedocs.io/en/latest/).


## Outline



- Link prediction task
- Graph classification task

In [8]:
# import the pytorch library into environment and check its version
import os
import torch
print("Using torch", torch.__version__)

Using torch 1.13.0+cu116


Let's start installing PyG by `pip`. The version of PyG should match the current version of PyTorch. Here we follow the [instruction](https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html) of PyG:

In [11]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install ogb  # for datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import some required libraries into our environment:

In [12]:
from torch_geometric.data import Data
from torch_geometric.datasets import SNAPDataset
from torch_geometric import nn
import torch_geometric.transforms as T


## Link prediction task


### Dataset preprocess

As shown in the following figure, link prediction is to predict whether two nodes in a graph have a link, which can be considered as a binary classification task. We will construct a link prediction dataset containing training, validation, and test set based on Cora. 


<br/>
<center>
<img src="https://github.com/Graph-and-Geometric-Learning/CPSC483-colab/blob/main/fig/link_prediction_example.png?raw=1" height="200" width="200"/>
</center>
<br/>


Given a graph, we divide the initial edge set into three distinct edge sets which represent the training, validation, and test set. Training set and validation set share a same graph structure. Test set contains some edges which does not exist in training and validation set to prevent data leakage.
<!-- Training set does not include edges in validation and test set, and the validation split does not include edges in the test split. Validation and test data should not be leaked into the training set. -->


<br/>
<center>
<img src="https://github.com/Graph-and-Geometric-Learning/CPSC483-colab/blob/main/fig/link_prediction_dataset_split(2).png?raw=1" height="200" width="350"/>
</center>
<br/>


Our model will be optimized on the training set. We can use `transforms` function in PyG to easily generate the data splits:

In [31]:
transform = T.Compose([
    T.RandomLinkSplit(num_val=0.05,  # ratio of edges including in the validation set
                      num_test=0.2,  # ratio of edges including in the test set
                      is_undirected=True,
                      add_negative_train_samples=False),
])


Loading the Cora dataset:

In [32]:
dataset = SNAPDataset('/tmp/snap', 'ego-facebook', transform=transform)

The data will be transformed from a data object to three tuples, where each element represents the corresponding split:

In [33]:
train_data, val_data, test_data = dataset[0]
print(train_data, val_data, test_data, sep="\n")

EgoData(x=[347, 1406], edge_index=[2, 4292], circle=[325], circle_batch=[325], edge_label=[2146], edge_label_index=[2, 2146])
EgoData(x=[347, 1406], edge_index=[2, 4292], circle=[325], circle_batch=[325], edge_label=[284], edge_label_index=[2, 284])
EgoData(x=[347, 1406], edge_index=[2, 4576], circle=[325], circle_batch=[325], edge_label=[1142], edge_label_index=[2, 1142])


Now data object has two attributes of edge: `edge_index` and `edge_label_index`. `edge_index` denotes the graph structure used for performing message passing in GNN. `edge_label_index` denotes the edge index used to calculate loss in training set, or to evaluate the model in validation and test set.


Printing the statistics of data:

In [34]:
print("Number of the nodes in training, validation and test data are", train_data.num_nodes, val_data.num_nodes, test_data.num_nodes)
print("Number of the edges in training, validation and test data are", train_data.num_edges, val_data.num_edges, test_data.num_edges)
print("Number of the edge_label_index in training, validation and test data are", train_data.edge_label_index.shape[1], 
                                                                                  val_data.edge_label_index.shape[1],
                                                                                  test_data.edge_label_index.shape[1])

Number of the nodes in training, validation and test data are 347 347 347
Number of the edges in training, validation and test data are 4292 4292 4576
Number of the edge_label_index in training, validation and test data are 2146 284 1142


### Pipeline

We constructed the GCN by PyG in the last Colab, and now we simply use the same architecture:

In [35]:
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()

        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.act = torch.nn.ReLU()

    def forward(self, node_feature, edge_index):

        output = self.conv1(node_feature, edge_index)
        output = self.act(output)
        output = self.conv2(output, edge_index)

        return output

Initializing a GCN model:

In [36]:
model = GCN(dataset.num_features, hidden_channels=128, out_channels=64)

Define an optimizer for model:

In [37]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

Similar as the what we do in the node classification task, we first apply the GCN model to produce the representation of each node in the graph. Usually we will use **inner product** to measure the similarity between two node representations to determine how likely it is for these two nodes to be connected.

#### Question 1

Following the instruction and implement the function to calculate the inner product:

In [38]:
def compute_similarity(node_embs, edge_index):
    result = torch.tensor([0.] * len(edge_index[0]))

    # TODO: Define similarity function.
    # 1. calculate the inner product between all the pairs in the edge_index
    # Note: the shape of node_embs is [n, h] where n is the number of nodes, and h is the embedding size
    # the shape of edge_index is [2, m] where m is the number of edges

    ############# Your code here ############
    ## (~1 line of code)
    node_embs = torch.nn.functional.normalize(node_embs)
    for i in range(len(edge_index[0])):
      result[i] = torch.dot(node_embs[edge_index[0][i]], node_embs[edge_index[1][i]])

    #########################################

    return result

n, h = 5, 10  # number of nodes and embedding size
node_embs = torch.rand(n, h)
edge_index = torch.tensor([[0, 1, 2, 3], 
                           [2, 3, 0, 1]])  # compute the similarity of (0, 2), (1, 3), (2, 0), (3, 1)
similarity = compute_similarity(node_embs, edge_index)
print("Similairty:", similarity)

Similairty: tensor([0.7905, 0.8211, 0.7905, 0.8211])


We optimize the model by minimizing the loss function. Here we consider the link prediction task as a binary classification task (edge exists or no), and apply binary cross entropy loss:

In [39]:
loss_fn = torch.nn.BCEWithLogitsLoss()

The edges in the graph will be taken as the positive examples with label=1 in the loss function. To prevent model from collapse, we usually will feed some **negative examples** to the loss function, which is the non-existing edges in the graph. The number of negative examples should equal to the number of positive ones.

With the help of PyG, we can easily perform the negative sampling. Here is an example:

In [40]:
from torch_geometric.utils import negative_sampling

neg_edge_index = negative_sampling(
      edge_index=train_data.edge_index,  # positive edges in the graph
      num_nodes=train_data.num_nodes,  # number of nodes
      num_neg_samples=5,  # number of negative examples
    )

print("shape of neg_edge_index:", neg_edge_index.shape)  # [2, num_neg_samples]
print("negative examples:", neg_edge_index)

shape of neg_edge_index: torch.Size([2, 5])
negative examples: tensor([[112,  79, 188, 227, 288],
        [ 74,  42, 198, 190, 128]])


Positive examples (`edge_label_index`) will be assigned the label 1, and negative ones will be assigned the label 0. We can obtain the label of positive examples like this:

In [41]:
print("positive examples' labels:", train_data.edge_label)

positive examples' labels: tensor([1., 1., 1.,  ..., 1., 1., 1.])


Now we can construct training and testing pipeline, which is similar to what we do in the last Colab. 

#### Question 2

Please follow the instruction and implement a function that trains a model.

In [45]:
def train(model, data, optimizer, loss_fn):

    loss = 0

    # TODO: Define train function.
    # 1. put the model into train mode
    # 2. clear the gradients calculated from the last batch
    # 3. use 'edge_index' to get the node representation by model
    # 4. sample the negative examples with the same number of positive ones (edge_label_index)
    # 5. concatenate the positive edges and negative edges
    # 6. concatenate the labels of positive edges and negative edges
    # 7. calculate the similarity between two end nodes to determine the probability that the corresponding edge is present on the graph.
    # 8. feed the probability and edge label to the loss function
    # 9. calculate the gradients of each parameter
    # 10. update the parameters by taking an optimizer step

    ############# Your code here ############
    ## (~10 line of code)
    model.train()
    optimizer.zero_grad()
    y_pred = model(data.x, data.edge_index)
    
    neg_edge_index = negative_sampling(
      edge_index=train_data.edge_index,  # positive edges in the graph
      num_nodes=train_data.num_nodes,  # number of nodes
      num_neg_samples=len(data.edge_label),  # number of negative examples
    )
    # print(neg_edge_index.shape)
    train_edge_index = torch.cat((data.edge_label_index, neg_edge_index), dim=1)
    train_edge_label = torch.cat((data.edge_label, torch.tensor([0] * len(train_data.edge_label_index[0]))))
    # print(train_edge_index.shape)
    pred = compute_similarity(data.x, train_edge_index)
    loss = loss_fn(pred.clone().detach().requires_grad_(True), train_edge_label)
    loss.backward()
    optimizer.step()

    #########################################

    return loss

We usually use [AUC score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html) to evaluate the performance of model on binary classification task. The test function is as followed:

In [46]:
from sklearn.metrics import roc_auc_score

@torch.no_grad()
def test(model, data):
    model.eval()
    out = model(data.x, data.edge_index)  # use `edge_index` to perform message passing
    out = compute_similarity(out, data.edge_label_index).view(-1).sigmoid()  # use `edge_label_index` to compute the loss
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

Now we can start to train our model based on `train` and `test` function:

In [47]:
epochs = 50

best_val_auc = final_test_auc = 0
for epoch in range(1, epochs + 1):
    loss = train(model, train_data, optimizer, loss_fn)
    valid_auc = test(model, val_data)
    test_auc = test(model, test_data)
    if valid_auc > best_val_auc:
        best_val_auc = valid_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {valid_auc:.4f}, Test: {test_auc:.4f}')

Epoch: 001, Loss: 0.7026, Val: 0.9423, Test: 0.9048
Epoch: 002, Loss: 0.7023, Val: 0.9423, Test: 0.9048
Epoch: 003, Loss: 0.7020, Val: 0.9423, Test: 0.9048
Epoch: 004, Loss: 0.7004, Val: 0.9423, Test: 0.9048
Epoch: 005, Loss: 0.7021, Val: 0.9423, Test: 0.9048
Epoch: 006, Loss: 0.7004, Val: 0.9423, Test: 0.9048
Epoch: 007, Loss: 0.7039, Val: 0.9423, Test: 0.9048
Epoch: 008, Loss: 0.7039, Val: 0.9423, Test: 0.9048
Epoch: 009, Loss: 0.7016, Val: 0.9423, Test: 0.9048
Epoch: 010, Loss: 0.7032, Val: 0.9423, Test: 0.9048
Epoch: 011, Loss: 0.7018, Val: 0.9423, Test: 0.9048
Epoch: 012, Loss: 0.7019, Val: 0.9423, Test: 0.9048
Epoch: 013, Loss: 0.7032, Val: 0.9423, Test: 0.9048
Epoch: 014, Loss: 0.7013, Val: 0.9423, Test: 0.9048
Epoch: 015, Loss: 0.7022, Val: 0.9423, Test: 0.9048
Epoch: 016, Loss: 0.7030, Val: 0.9423, Test: 0.9048
Epoch: 017, Loss: 0.7025, Val: 0.9423, Test: 0.9048
Epoch: 018, Loss: 0.7023, Val: 0.9423, Test: 0.9048
Epoch: 019, Loss: 0.7020, Val: 0.9423, Test: 0.9048
Epoch: 020, 